## Create Myanmar maps with reservoirs

### T. Janus
tomasz.k.janus@gmail.com

2. ### Load libraries and data

In [ ]:
library(tidyverse)
library(dplyr)
library(sf)
library(Hmisc)
library(ggplot2)
library(colorspace)
library(ggspatial)

In [ ]:
mya_epsg_code <- "EPSG:32646"
googlecrs <- "EPSG:4326"

In [ ]:
mya_outline <- sf::st_read(
    dsn = file.path("bin","gis_layers","myanmar_outline","Myanmar_outline.shp"), 
    quiet = TRUE, geometry_column = 'geometry') %>% 
    st_transform(mya_epsg_code)

df_sf <- sf::st_as_sf(mya_outline, coords=c("Lon", "Lat"), crs=googlecrs, agr = "identity")
df_xy <- sf::st_transform(df_sf, crs=mya_epsg_code)
bbox <- sf::st_bbox(df_xy)
# Expand box by 20% to give a little extra room
Dx <- (bbox[["xmax"]]-bbox[["xmin"]])*0.1
Dy <- (bbox[["ymax"]]-bbox[["ymin"]])*0.1
bbox["xmin"] <- bbox["xmin"] - Dx
bbox["xmax"] <- bbox["xmax"] + Dx
bbox["ymin"] <- bbox["ymin"] - Dy
bbox["ymax"] <- bbox["ymax"] + Dy
# 
bb <- c(bbox["xmin"], bbox["ymin"], bbox["xmax"], bbox["ymax"])

In [ ]:
# Load and filter shape data
mya_reservoirs <- sf::st_read(
    dsn = file.path("bin", "heet_outputs_MIN_LOW_PRI","reservoirs_updated.shp"), quiet = TRUE) %>% 
    st_transform(mya_epsg_code) %>% 
    filter(!is.na(tot_em)) %>% 
    mutate(type = replace(type, type == "unknown", "hydroelectric")) %>%
    rename(reservoir_type = type) %>%
    # Get some names in mya_reservoirs to the IFC format (some names are off as they were not coded properly in HEET)
    mutate(name = case_when(
        name == "Paung Laung (lower)" ~ "Lower Paunglaung", 
        name == "Paung Laung (upper)" ~ "Upper Paunglaung",
        name == "Thapanzeik" ~ "Thaphanseik",
        TRUE ~ name
    ))

res_centroids <- sf::st_centroid(mya_reservoirs, of_largest_polygon = TRUE) %>% arrange(desc(r_area_km2))
mya_rivers <- sf::st_read(dsn = file.path("bin","gis_layers","mya_rivers.shp"), quiet = TRUE)  %>% 
    st_transform(mya_epsg_code)
mya_neighbours <- sf::st_read(
    dsn = file.path("bin","gis_layers","World_Countries_Generalized.shp"), quiet = TRUE)  %>% 
    st_transform(mya_epsg_code) %>%
    filter(COUNTRY != "Myanmar")
mya_neighbours_crop <- st_crop(mya_neighbours, y = bbox)

### 2. CREATE A FACET MAP OF EMISSION INTENSITIES WITH ALL THREE RESERVOIR TYPES

#### Instructions
* to remove basenap comment out `ggspatial::annotation_map_tile(
        type = "osm", zoom = 6, alpha=0.4, interpolate = TRUE, quiet = TRUE) +`
* increase alpha in `geom_sf(data=mya_neighbours_crop, fill="lightgrey", col="black", alpha=0.1) +` from 0.1 to 0.5
* increase alpha in `geom_sf(data=mya_outline, fill="antiquewhite1", col="black", alpha=0.8, size = 5.5)` to 0.7 to 0.8

### 2A. Unit emissions per area

In [ ]:
# Bin emissions
em_ranges <- c(
    "< 500", 
    "500 - 1000", 
    "1000 - 2000", 
    "2000 - 3000",
    "3000 - 5000",
    "> 5000"
)
res_centroids <- 
    res_centroids %>%
    mutate(
        em_range = factor(case_when(
            `tot_em` < 500 ~ em_ranges[1],
            between(`tot_em`, 500, 1000) ~ em_ranges[2],
            between(`tot_em`, 1000, 2000) ~ em_ranges[3],
            between(`tot_em`, 2000, 3000) ~ em_ranges[4],
            between(`tot_em`, 3000, 5000) ~ em_ranges[5],
            `tot_em` >= 5000 ~ em_ranges[6]
        ), levels = em_ranges)
    )

#Named 'palette' should be one of: Blue-Red, Blue-Red 2, Blue-Red 3, Red-Green, Purple-Green, 
#Purple-Brown, Green-Brown, Blue-Yellow 2, Blue-Yellow 3, Green-Orange, Cyan-Magenta, Tropic, 
#Broc, Cork, Vik, Berlin, Lisbon, Tofino
colors_map <- colorspace::diverging_hcl(6, palette = "Tropic", rev = F) # OK: Cork

mya_gg <- ggplot(lims_method = "geometry_bbox")
mya_gg +
    geom_sf(fill = "lightgrey", alpha=0.9) + #fill = "antiquewhite1"
    #coord_equal(xlim = c(0, 28), ylim = c(0, 20), expand = FALSE) + 
    #ggspatial::annotation_map_tile(
    #    type = "osm", zoom = 6, alpha=0.4, interpolate = TRUE, quiet = TRUE) +
    geom_sf(data=mya_neighbours_crop, fill="lightgrey", col="black", alpha=0.5) +
    geom_sf(data=mya_outline, fill="antiquewhite1", col="black", alpha=0.8, size = 5.5) +
    #geom_sf(data=mya_reservoirs, col="grey") +
    geom_sf_label(data=mya_neighbours_crop, aes(label = COUNTRY), fill = "white") + 
    #geom_sf(data = st_as_sfc(bbox), fill = NA, color = "red", size = 1) +
    geom_sf(data=mya_rivers, col="lightblue3") +
    geom_sf(data = res_centroids %>% arrange(desc(r_area_km2)),
          pch = 21,
          aes(size = r_area_km2, fill = em_range),
          col = "Gray60") +
    scale_size(breaks = c(10, 50, 100, 300, 500),
        range = c(1, 15),
        guide = guide_legend(
          direction = "horizontal",
          ncol = 5)) +
    scale_fill_manual(values = colors_map, name="Emission intensity\ngCO2eq/m2/year") +
    #scale_fill_continuous(name="Emission intensity",limits = c(0,5000),
    #    breaks=c(300, 1000,2000,3000, 5000), 
    #    low = "white", high = "darkred") +
    #scale_fill_gradientn(colours = hcl.colors(n=100, palette = "RdBu", fixup = 4,
    #                                            rev = TRUE,
    #                                            alpha = 0.7)) +
    #guides(fill = guide_colorbar(title = "Emission intensity\ngCO2eq/m2/year")) +
    labs(title = "GHG emission intensities per reservoir type",
           sub = "FutureDAMS Project",
           #caption = "T. Janus, 2023",
           x = NULL, y = NULL,
           size = "Reservoir Area\nkm2") +
    xlab(NULL) + ylab(NULL) +
    annotate(geom = "text", x = 820000, y = 1500000, label = "Andaman\nSea", 
     fontface = "italic", color = "grey22", size = 4) +
    annotate(geom = "text", x = 505000, y = 1950000, label = "Bay\nof\nBengal", 
     fontface = "italic", color = "grey22", size = 4) +
    ggspatial::annotation_scale(
        location = "tl",
        bar_cols = c("grey60", "white"),
        text_family = "Avenir Next Condensed") +
    ggspatial::annotation_north_arrow(
        location = "tl", which_north = "true",
        pad_x = unit(0.12, "in"), pad_y = unit(0.3, "in"),
        style = ggspatial::north_arrow_fancy_orienteering(
          fill = c("grey40", "white"),
          line_col = "grey20",
          text_family = "Avenir Next Condensed"))+
    facet_wrap(~reservoir_type) +
    xlim(bbox["xmin"]+Dx, bbox["xmax"]-Dx) + ylim(bbox["ymin"]+Dy, bbox["ymax"]-Dy) +
    #geom_segment(aes(x = x1, y = y1, xend = x2, yend = y2), data = arrowA, 
    #    arrow = arrow(), lineend = "round") +
    #geom_segment(aes(x = x1, y = y1, xend = x2, yend = y2), data = arrowB, 
    #    arrow = arrow(), lineend = "round") +
    #geom_text( data=mya_reservoirs, hjust=0.5, vjust=-0.5, aes(x=long, y=lat, label=name), colour="gold2", size=4 ) +
    coord_sf(crs = mya_epsg_code) +
    theme_minimal() +
    theme(
        text = element_text("Avenir Next Condensed"),
        strip.text = element_text(face="bold", hjust = 0),
        plot.title = element_text(hjust = 0),
        plot.caption = element_text(face = "italic"),
        panel.grid.major = element_line(
            colour = gray(0.5), linetype = "dashed", size = 0.5), 
        panel.background = element_rect(fill = "aliceblue"), 
        panel.border = element_rect(fill = NA),
        legend.direction = "horizontal", legend.position = "bottom",
        panel.ontop = FALSE
    )
ggsave(
    file.path("figures","maps","emission_intensities_per_res_type.png"), 
    width = 27, height = 20, units = "cm")
#ggsave("map-us-ggdraw.pdf", width = 5, height = 8)
dev.off()

### 2B. Total emissions

In [ ]:
# Calculate total emissions (in thousands of tonnes CO2eq/year)
res_centroids$tot_em_int = res_centroids$tot_em * res_centroids$r_area_km2 / 1000

In [ ]:
Hmisc::describe(res_centroids$tot_em_int)

In [ ]:
# Bin emissions based on total emissions in thousands of tonnes CO2eq/year
tot_em_ranges <- c(
    "< 3", 
    "3 - 15", 
    "15 - 30", 
    "30 - 50",
    "50 - 200",
    "> 200"
)

res_centroids <- 
    res_centroids %>%
    mutate(
        tot_em_range = factor(case_when(
            `tot_em_int` < 3 ~ tot_em_ranges[1],
            between(`tot_em_int`, 3, 15) ~ tot_em_ranges[2],
            between(`tot_em_int`, 15, 30) ~ tot_em_ranges[3],
            between(`tot_em_int`, 30, 50) ~ tot_em_ranges[4],
            between(`tot_em_int`, 50, 200) ~ tot_em_ranges[5],
            `tot_em_int` >= 200 ~ tot_em_ranges[6]
        ), levels = tot_em_ranges)
    )

In [ ]:
#Named 'palette' should be one of: Blue-Red, Blue-Red 2, Blue-Red 3, Red-Green, Purple-Green, 
#Purple-Brown, Green-Brown, Blue-Yellow 2, Blue-Yellow 3, Green-Orange, Cyan-Magenta, Tropic, 
#Broc, Cork, Vik, Berlin, Lisbon, Tofino
colors_map <- colorspace::diverging_hcl(6, palette = "Tropic", rev = F) # OK: Cork

mya_gg <- ggplot(lims_method = "geometry_bbox")
mya_gg +
    geom_sf(fill = "lightgrey", alpha=0.9) + #fill = "antiquewhite1"
    #coord_equal(xlim = c(0, 28), ylim = c(0, 20), expand = FALSE) + 
    #ggspatial::annotation_map_tile(
    #    type = "osm", zoom = 6, alpha=0.4, interpolate = TRUE, quiet = TRUE) +
    geom_sf(data=mya_neighbours_crop, fill="lightgrey", col="black", alpha=0.5) +
    geom_sf(data=mya_outline, fill="antiquewhite1", col="black", alpha=0.8, size = 5.5) +
    #geom_sf(data=mya_reservoirs, col="grey") +
    geom_sf_label(data=mya_neighbours_crop, aes(label = COUNTRY), fill = "white") + 
    #geom_sf(data = st_as_sfc(bbox), fill = NA, color = "red", size = 1) +
    geom_sf(data=mya_rivers, col="lightblue3") +
    geom_sf(data = res_centroids %>% arrange(desc(r_area_km2)),
          pch = 21,
          aes(size = r_area_km2, fill = tot_em_range),
          col = "Gray60") +
    scale_size(breaks = c(10, 50, 100, 300, 500),
        range = c(1, 15),
        guide = guide_legend(
          direction = "horizontal",
          ncol = 5)) +
    scale_fill_manual(values = colors_map, name="Total emission\nktonneCO2eq/year") +
    #scale_fill_continuous(name="Emission intensity",limits = c(0,5000),
    #    breaks=c(300, 1000,2000,3000, 5000), 
    #    low = "white", high = "darkred") +
    #scale_fill_gradientn(colours = hcl.colors(n=100, palette = "RdBu", fixup = 4,
    #                                            rev = TRUE,
    #                                            alpha = 0.7)) +
    #guides(fill = guide_colorbar(title = "Emission intensity\ngCO2eq/m2/year")) +
    labs(title = "GHG emissions per reservoir type",
           sub = "FutureDAMS Project",
           #caption = "T. Janus, 2023",
           size = "Reservoir Area\nkm2") +
    xlab(NULL) + ylab(NULL) +
    theme_minimal() +
    theme(
        text = element_text("Avenir Next Condensed"),
        strip.text = element_text(face="bold", hjust = 0),
        plot.title = element_text(hjust = 0),
        plot.caption = element_text(face = "italic"),
        panel.grid.major = element_line(
            colour = gray(0.5), linetype = "dashed", size = 0.5), 
        panel.background = element_rect(fill = "aliceblue"), 
        panel.border = element_rect(fill = NA),
        legend.direction = "horizontal", legend.position = "bottom",
        panel.ontop = FALSE
    ) +
    annotate(geom = "text", x = 820000, y = 1500000, label = "Andaman\nSea", 
     fontface = "italic", color = "grey22", size = 4) +
    annotate(geom = "text", x = 505000, y = 1950000, label = "Bay\nof\nBengal", 
     fontface = "italic", color = "grey22", size = 4) +
    ggspatial::annotation_scale(
        location = "tl",
        bar_cols = c("grey60", "white"),
        text_family = "ArcherPro Book") +
    ggspatial::annotation_north_arrow(
        location = "tl", which_north = "true",
        pad_x = unit(0.12, "in"), pad_y = unit(0.3, "in"),
        style = ggspatial::north_arrow_fancy_orienteering(
          fill = c("grey40", "white"),
          line_col = "grey20",
          text_family = "ArcherPro Book"))+
    facet_wrap(~reservoir_type) +
    xlim(bbox["xmin"]+Dx, bbox["xmax"]-Dx) + ylim(bbox["ymin"]+Dy, bbox["ymax"]-Dy) +
    #geom_segment(aes(x = x1, y = y1, xend = x2, yend = y2), data = arrowA, 
    #    arrow = arrow(), lineend = "round") +
    #geom_segment(aes(x = x1, y = y1, xend = x2, yend = y2), data = arrowB, 
    #    arrow = arrow(), lineend = "round") +
    #geom_text( data=mya_reservoirs, hjust=0.5, vjust=-0.5, aes(x=long, y=lat, label=name), colour="gold2", size=4 ) +
    coord_sf(crs = mya_epsg_code)
ggsave(
    file.path("figures","maps", "total_emissions_per_res_type.png"), width = 27, height = 20, units = "cm")
#ggsave("map-us-ggdraw.pdf", width = 5, height = 8)
dev.off()

## Plot a minimal map for visual abstract

In [ ]:
#Named 'palette' should be one of: Blue-Red, Blue-Red 2, Blue-Red 3, Red-Green, Purple-Green, 
#Purple-Brown, Green-Brown, Blue-Yellow 2, Blue-Yellow 3, Green-Orange, Cyan-Magenta, Tropic, 
#Broc, Cork, Vik, Berlin, Lisbon, Tofino
colors_map <- colorspace::diverging_hcl(6, palette = "Tropic", rev = F) # OK: Cork

mya_gg <- ggplot(lims_method = "geometry_bbox")
mya_gg +
    geom_sf(fill = "lightgrey", alpha=0.9) + #fill = "antiquewhite1"
    #coord_equal(xlim = c(0, 28), ylim = c(0, 20), expand = FALSE) + 
    #ggspatial::annotation_map_tile(
    #    type = "osm", zoom = 6, alpha=0.4, interpolate = TRUE, quiet = TRUE) +
    #geom_sf(data=mya_neighbours_crop, fill="lightgrey", col="black", alpha=0.5) +
    geom_sf(data=mya_outline, fill="antiquewhite1", col="black", alpha=1, size = 5.5) +
    #geom_sf(data = st_as_sfc(bbox), fill = NA, color = "red", size = 1) +
    geom_sf(data=mya_rivers, col="lightblue3") +
    geom_sf(data = res_centroids,
          pch = 24,
          size=2.5,
          alpha = 0.7,
          aes(fill = tot_em_range),
          col = "black") +

    #scale_size_manual(values = c(1,2,3,4,5,6,7,8)) +
    scale_fill_manual(values = colors_map, name="Total emission\nktonneCO2eq/year") +
    #scale_fill_continuous(name="Emission intensity",limits = c(0,5000),
    #    breaks=c(300, 1000,2000,3000, 5000), 
    #    low = "white", high = "darkred") +
    #scale_fill_gradientn(colours = hcl.colors(n=100, palette = "RdBu", fixup = 4,
    #                                            rev = TRUE,
    #                                            alpha = 0.7)) +
    #guides(fill = guide_colorbar(title = "Emission intensity\ngCO2eq/m2/year")) +
    labs(#caption = "T. Janus, 2023",
           size = "Reservoir Area\nkm2") +
    xlab(NULL) + ylab(NULL) +
    theme_void() +
    theme(
        text = element_text("Avenir Next Condensed"),
        strip.text = element_text(face="bold", hjust = 0),
        plot.title = element_text(hjust = 0),
        plot.caption = element_text(face = "italic"),
        #panel.grid.major = element_line(
        #    colour = gray(0.5), linetype = "dashed", size = 0.5), 
        panel.background = element_blank(),
        #panel.background = element_rect(fill = NA), 
        panel.border = element_blank(),
        legend.direction = "horizontal", legend.position = "none",
        panel.ontop = FALSE
    ) +
    xlim(bbox["xmin"]+Dx, bbox["xmax"]-Dx) + ylim(bbox["ymin"]+Dy, bbox["ymax"]-Dy) +
    #geom_segment(aes(x = x1, y = y1, xend = x2, yend = y2), data = arrowA, 
    #    arrow = arrow(), lineend = "round") +
    #geom_segment(aes(x = x1, y = y1, xend = x2, yend = y2), data = arrowB, 
    #    arrow = arrow(), lineend = "round") +
    #geom_text( data=mya_reservoirs, hjust=0.5, vjust=-0.5, aes(x=long, y=lat, label=name), colour="gold2", size=4 ) +
    coord_sf(crs = mya_epsg_code)
ggsave(
    file.path("figures","maps","total_emissions_vis_abstract.png"), width = 27, height = 20, units = "cm")
#ggsave("map-us-ggdraw.pdf", width = 5, height = 8)
dev.off()

### 2C. Total emissions divided by existing and future dams

In [ ]:
### Import ifc database that contains information about statuses of hydroelectric and multipurpose dams
ifc_dams <- sf::st_read(
    dsn = file.path("bin", "gis_layers", "ifc_database", "all_dams_replaced_refactored.shp"), 
    quiet = TRUE)  %>% 
    st_transform(mya_epsg_code) %>%
    filter(Status != "Built") %>%
    st_drop_geometry() %>%
    select("IFC_ID", "Status", "DAM_NAME") %>%
    mutate(Status = ifelse(
        Status %in% c(
            'Construction', 'MOU', 'LocMoU', 'Identified', 'GOM Plan', 'Suspended', 'MOA',
            'Covenant', 'JVA'), 'Future', Status))
# Set status in res_centroids to Existing by default.
res_centroids$Status = "Existing"
# Join the centroids dataframe with ifc_dams and replace the status to Future for Future IFC dams
res_centroids_joined = dplyr::left_join(res_centroids, ifc_dams, by = join_by(id == IFC_ID)) %>%
  st_transform(mya_epsg_code) %>%
  mutate(Status = ifelse(is.na(Status.y), Status.x, Status.y)) %>%
  select(-Status.x, -Status.y) %>%
  mutate(Status = ifelse(
      Status == "Future", "Future Hydroelectric", Status)) %>%
  mutate(Status = ifelse(
      Status == "Existing", "Existing Reservoirs", Status)) %>%
  arrange(desc(r_area_km2))

# Find a list of future RoR hydroelectric from the IFC database

ifc_dams_ror_future <- sf::st_read(
    dsn = file.path("bin","gis_layers","ifc_database","all_dams_replaced_refactored.shp"), 
    quiet = TRUE) %>%
    st_transform(mya_epsg_code) %>%
    filter(Status != "Built") %>%
    mutate(`RoR.or.Sto` = tolower(`RoR.or.Sto`)) %>%
    filter(`RoR.or.Sto` == 'ror') %>%
    mutate(Status = "Future Hydroelectric")

ifc_dams_ror_existing <- sf::st_read(
    dsn = file.path("bin","gis_layers","ifc_database","all_dams_replaced_refactored.shp"), 
    quiet = TRUE) %>%
    st_transform(mya_epsg_code) %>%
    filter(Status == "Built") %>%
    mutate(`RoR.or.Sto` = tolower(`RoR.or.Sto`)) %>%
    filter(`RoR.or.Sto` == 'ror') %>%
    mutate(Status = "Existing Reservoirs")

ifc_dams_ror_all <- rbind(ifc_dams_ror_future, ifc_dams_ror_existing) 

Find if any future RoRs are already included in the reservoirs list for GHG emission estimation 
(some RoRs were misclassified as RoR whereas in fact they're small storage dams)

In [ ]:
doubled_names_all <- dplyr::inner_join(res_centroids_joined %>% 
    st_drop_geometry(), ifc_dams_ror_all, by=join_by(name == DAM_NAME)) %>%
    select(name) %>%
    as.list()

doubled_names_future <- dplyr::inner_join(res_centroids_joined %>% 
    st_drop_geometry(), ifc_dams_ror_future, by=join_by(name == DAM_NAME)) %>%
    select(name) %>%
    as.list()

In [ ]:
doubled_names_future

In [ ]:
ifc_dams_ror_all_filt <- ifc_dams_ror_all %>% filter(!(DAM_NAME %in% doubled_names_all$name))
ifc_dams_ror_future_filt <- ifc_dams_ror_future %>% filter(!(DAM_NAME %in% doubled_names_future$name))

In [ ]:
#Named 'palette' should be one of: Blue-Red, Blue-Red 2, Blue-Red 3, Red-Green, Purple-Green, 
#Purple-Brown, Green-Brown, Blue-Yellow 2, Blue-Yellow 3, Green-Orange, Cyan-Magenta, Tropic, 
#Broc, Cork, Vik, Berlin, Lisbon, Tofino
colors_map <- colorspace::diverging_hcl(7, palette = "Tropic", rev = F) # OK: Cork

mya_gg <- ggplot(lims_method = "geometry_bbox")
mya_gg +
    geom_sf(fill = "lightgrey", alpha=0.9) + #fill = "antiquewhite1"
    #coord_equal(xlim = c(0, 28), ylim = c(0, 20), expand = FALSE) + 
    #ggspatial::annotation_map_tile(
    #    type = "osm", zoom = 6, alpha=0.4, interpolate = TRUE, quiet = TRUE) +
    geom_sf(data=mya_neighbours_crop, fill="lightgrey", col="black", alpha=0.5) +
    geom_sf(data=mya_outline, fill="antiquewhite1", col="black", alpha=0.8, size = 5.5) +
    #geom_sf(data=mya_reservoirs, col="grey") +
    geom_sf_label(data=mya_neighbours_crop, aes(label = COUNTRY), fill = "white") + 
    #geom_sf(data = st_as_sfc(bbox), fill = NA, color = "red", size = 1) +
    geom_sf(data=mya_rivers, col="lightblue3") +
    geom_sf(data = res_centroids_joined,
          pch = 21,
          aes(size = r_area_km2, fill = tot_em_range),
          col = "Gray60") +
    scale_size(breaks = c(10, 50, 100, 300, 500),
        range = c(1, 15),
        guide = guide_legend(
          direction = "horizontal",
          ncol = 5)) +
    scale_fill_manual(values = colors_map, name="Total emission\nktonneCO2eq/year") +

    geom_sf(data=ifc_dams_ror_all_filt, pch = 24, color="black", fill="white", 
            alpha=0.8, size=1) +

    #scale_fill_continuous(name="Emission intensity",limits = c(0,5000),
    #    breaks=c(300, 1000,2000,3000, 5000), 
    #    low = "white", high = "darkred") +
    #scale_fill_gradientn(colours = hcl.colors(n=100, palette = "RdBu", fixup = 4,
    #                                            rev = TRUE,
    #                                            alpha = 0.7)) +
    #guides(fill = guide_colorbar(title = "Emission intensity\ngCO2eq/m2/year")) +
    labs(title = "GHG emissions per construction status",
           sub = "FutureDAMS Project",
           #caption = "T. Janus, 2023",
           size = "Reservoir Area\nkm2") +
    xlab(NULL) + ylab(NULL) +
    theme_minimal() +
    theme(
        text = element_text("Avenir Next Condensed"),
        strip.text = element_text(face="bold", hjust = 0),
        plot.title = element_text(hjust = 0),
        plot.caption = element_text(face = "italic"),
        panel.grid.major = element_line(
            colour = gray(0.5), linetype = "dashed", size = 0.5), 
        panel.background = element_rect(fill = "aliceblue"), 
        panel.border = element_rect(fill = NA),
        legend.direction = "horizontal", legend.position = "bottom",
        panel.ontop = FALSE
    ) +
    annotate(geom = "text", x = 820000, y = 1500000, label = "Andaman\nSea", 
     fontface = "italic", color = "grey22", size = 4) +
    annotate(geom = "text", x = 505000, y = 1950000, label = "Bay\nof\nBengal", 
     fontface = "italic", color = "grey22", size = 4) +
    ggspatial::annotation_scale(
        location = "tl",
        bar_cols = c("grey60", "white"),
        text_family = "ArcherPro Book") +
    ggspatial::annotation_north_arrow(
        location = "tl", which_north = "true",
        pad_x = unit(0.12, "in"), pad_y = unit(0.3, "in"),
        style = ggspatial::north_arrow_fancy_orienteering(
          fill = c("grey40", "white"),
          line_col = "grey20",
          text_family = "ArcherPro Book"))+
    facet_wrap(~Status) +
    xlim(bbox["xmin"]+Dx, bbox["xmax"]-Dx) + ylim(bbox["ymin"]+Dy, bbox["ymax"]-Dy) +
    #geom_segment(aes(x = x1, y = y1, xend = x2, yend = y2), data = arrowA, 
    #    arrow = arrow(), lineend = "round") +
    #geom_segment(aes(x = x1, y = y1, xend = x2, yend = y2), data = arrowB, 
    #    arrow = arrow(), lineend = "round") +
    #geom_text( data=mya_reservoirs, hjust=0.5, vjust=-0.5, aes(x=long, y=lat, label=name), colour="gold2", size=4 ) +
    coord_sf(crs = mya_epsg_code)
ggsave(
    file.path("figures","maps","total_emissions_per_status.png"), width = 27, height = 20, units = "cm")
#ggsave("map-us-ggdraw.pdf", width = 5, height = 8)
dev.off()

### 2D. Emissions per HP generated of future hydroelectric dams

In [ ]:
# Merge HP production figures into res_centroids data
emissions_data <- readxl::read_excel(
    file.path("intermediate", "merged_table.xlsx")) %>%
    distinct(ifc_id, .keep_all = TRUE) %>%
    select(
        ifc_id, dam_name, status_2_ifc, ro_r_or_sto_ifc, hp_type_reem, 
        res_area, mean, pctile_2, pctile_3, pctile_5, pctile_10) %>%
    rename(name=dam_name) %>%
    mutate(ann_gen = mean * 24 * 365.25 / 1000)

hp_dams <- res_centroids_joined %>%
    filter(reservoir_type!='irrigation')

joined_data <- merge(hp_dams, emissions_data, by = "name", all.x=TRUE) %>%
    mutate(em_intensity = tot_em_int / ann_gen * 1000) %>%
    mutate(Status = ifelse(Status == 'Existing Reservoirs', 'Existing Hydroelectric', Status))

In [ ]:
Hmisc::describe(joined_data$em_intensity)

In [ ]:
# Bin emissions intensities in tonnesCO2eq/GWh
em_intensity_ranges <- c(
    "< 20", 
    "20 - 50", 
    "50 - 150", 
    "150 - 300",
    "300 - 450",
    "450 - 1500",
    "> 1500"
)

joined_data <- 
    joined_data %>%
    mutate(
        em_intensity_range = factor(case_when(
            `em_intensity` < 20 ~ em_intensity_ranges[1],
            between(`em_intensity`, 20, 50) ~ em_intensity_ranges[2],
            between(`em_intensity`, 50, 150) ~ em_intensity_ranges[3],
            between(`em_intensity`, 150, 300) ~ em_intensity_ranges[4],
            between(`em_intensity`, 300, 450) ~ em_intensity_ranges[5],
            between(`em_intensity`, 450, 1500) ~ em_intensity_ranges[6],
            `em_intensity` >= 1500 ~ em_intensity_ranges[7]
        ), levels = em_intensity_ranges)
    )

In [ ]:
hist(joined_data$em_intensity)

In [ ]:
#Named 'palette' should be one of: Blue-Red, Blue-Red 2, Blue-Red 3, Red-Green, Purple-Green, 
#Purple-Brown, Green-Brown, Blue-Yellow 2, Blue-Yellow 3, Green-Orange, Cyan-Magenta, Tropic, 
#Broc, Cork, Vik, Berlin, Lisbon, Tofino
colors_map <- colorspace::diverging_hcl(7, palette = "Tropic", rev = F) # OK: Cork

ifc_dams_ror_all_filt2 <- ifc_dams_ror_all_filt %>%
    mutate(Status = case_when(
        Status == "Existing Reservoirs" ~ "Existing Hydroelectric", 
        TRUE ~ Status
    ))

mya_gg <- ggplot(lims_method = "geometry_bbox")
mya_gg +
    geom_sf(fill = "lightgrey", alpha=0.9) + #fill = "antiquewhite1"
    #coord_equal(xlim = c(0, 28), ylim = c(0, 20), expand = FALSE) + 
    #ggspatial::annotation_map_tile(
    #    type = "osm", zoom = 6, alpha=0.4, interpolate = TRUE, quiet = TRUE) +
    geom_sf(data=mya_neighbours_crop, fill="lightgrey", col="black", alpha=0.5) +
    geom_sf(data=mya_outline, fill="antiquewhite1", col="black", alpha=0.8, size = 5.5) +
    #geom_sf(data=mya_reservoirs, col="grey") +
    geom_sf_label(data=mya_neighbours_crop, aes(label = COUNTRY), fill = "white") + 
    #geom_sf(data = st_as_sfc(bbox), fill = NA, color = "red", size = 1) +
    geom_sf(data=mya_rivers, col="lightblue3") +
    geom_sf(data = joined_data %>% arrange(desc(r_area_km2)),
          pch = 21,
          aes(size = r_area_km2, fill = em_intensity_range),
          col = "Gray60") +
    scale_size(breaks = c(10, 50, 100, 300, 500),
        range = c(1, 15),
        guide = guide_legend(
          direction = "horizontal",
          ncol = 3)) +
    scale_fill_manual(
        values = colors_map, 
        name="Emission footprint\ntonne CO2eq/GWh",
        guide = guide_legend(
          direction = "horizontal",
          ncol = 2)) +

    geom_sf(data=ifc_dams_ror_all_filt2, pch = 24, color="black", fill="white", alpha=0.8, size=1.3) +

    #scale_fill_continuous(name="Emission intensity",limits = c(0,5000),
    #    breaks=c(300, 1000,2000,3000, 5000), 
    #    low = "white", high = "darkred") +
    #scale_fill_gradientn(colours = hcl.colors(n=100, palette = "RdBu", fixup = 4,
    #                                            rev = TRUE,
    #                                            alpha = 0.7)) +
    #guides(fill = guide_colorbar(title = "Emission intensity\ngCO2eq/m2/year")) +
    labs(title = expression(paste("GHG emissions per generated energy in dams over 5 M", m^3)),
           sub = "FutureDAMS Project",
           #caption = "T. Janus, 2023",
           size = "Reservoir Area\nkm2") +
    xlab(NULL) + ylab(NULL) +
    theme_minimal() +
    theme(
        text = element_text("Avenir Next Condensed"),
        strip.text = element_text(face="bold", hjust = 0),
        plot.title = element_text(hjust = 0),
        plot.caption = element_text(face = "italic"),
        panel.grid.major = element_line(
            colour = gray(0.5), linetype = "dashed", size = 0.5), 
        panel.background = element_rect(fill = "aliceblue"), 
        panel.border = element_rect(fill = NA),
        legend.direction = "horizontal", legend.position = "bottom",
        panel.ontop = FALSE
    ) +
    annotate(geom = "text", x = 820000, y = 1500000, label = "Andaman\nSea", 
     fontface = "italic", color = "grey22", size = 4) +
    annotate(geom = "text", x = 505000, y = 1950000, label = "Bay\nof\nBengal", 
     fontface = "italic", color = "grey22", size = 4) +
    ggspatial::annotation_scale(
        location = "tl",
        bar_cols = c("grey60", "white"),
        text_family = "ArcherPro Book") +
    ggspatial::annotation_north_arrow(
        location = "tl", which_north = "true",
        pad_x = unit(0.12, "in"), pad_y = unit(0.3, "in"),
        style = ggspatial::north_arrow_fancy_orienteering(
          fill = c("grey40", "white"),
          line_col = "grey20",
          text_family = "ArcherPro Book"))+
    facet_wrap(~Status) +
    xlim(bbox["xmin"]+Dx, bbox["xmax"]-Dx) + ylim(bbox["ymin"]+Dy, bbox["ymax"]-Dy) +
    #geom_segment(aes(x = x1, y = y1, xend = x2, yend = y2), data = arrowA, 
    #    arrow = arrow(), lineend = "round") +
    #geom_segment(aes(x = x1, y = y1, xend = x2, yend = y2), data = arrowB, 
    #    arrow = arrow(), lineend = "round") +
    #geom_text( data=mya_reservoirs, hjust=0.5, vjust=-0.5, aes(x=long, y=lat, label=name), colour="gold2", size=4 ) +
    coord_sf(crs = mya_epsg_code)
ggsave(
    file.path("figures","maps","total_emissions_per_GWh.png"), width = 27, height = 20, units = "cm")
#ggsave("map-us-ggdraw.pdf", width = 5, height = 8)
dev.off()

#length(joined_data$em_intensity)